<a href="https://colab.research.google.com/github/vanderbilt-ml/51-assign05-BRay/blob/main/Assignment05_bray_hof_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basketball Hall of Fame

Imports

In [214]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [231]:
#tables and visualizations
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#machine learning
from sklearn.ensemble import GradientBoostingClassifier, RandomTreesEmbedding #Gradient Model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

employee_df = pd.read_csv('seasons.csv')
display(employee_df.head())
employee_df.info()

In [246]:
#employee_df = pd.read_csv('./MyDrive/csv/seasons.csv')
#display(employee_df.head())
#employee_df.info()


path="/content/drive/MyDrive/csv/seasons_stats.csv";
hof_df=pd.read_csv(path)

display(hof_df.head())
hof_df.info()

,Year,HOF,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1986.0,No,A.C. Green,PF,22.0,LAL,82.0,1.0,1542.0,11.8,...,0.611,160.0,221.0,381.0,54.0,49.0,49.0,99.0,229.0,521.0
1,1987.0,No,A.C. Green,PF,23.0,LAL,79.0,72.0,2240.0,15.7,...,0.780,210.0,405.0,615.0,84.0,70.0,80.0,102.0,171.0,852.0
2,1988.0,No,A.C. Green,PF,24.0,LAL,82.0,64.0,2636.0,14.5,...,0.773,245.0,465.0,710.0,93.0,87.0,45.0,120.0,204.0,937.0
3,1989.0,No,A.C. Green,PF,25.0,LAL,82.0,82.0,2510.0,17.8,...,0.786,258.0,481.0,739.0,103.0,94.0,55.0,119.0,172.0,1088.0
4,1990.0,No,A.C. Green,PF,26.0,LAL,82.0,82.0,2709.0,14.7,...,0.751,262.0,450.0,712.0,90.0,66.0,50.0,116.0,207.0,1061.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24691 entries, 0 to 24690
Data columns (total 51 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    24624 non-null  float64
 1   HOF     24679 non-null  object 
 2   Player  24624 non-null  object 
 3   Pos     24624 non-null  object 
 4   Age     24616 non-null  float64
 5   Tm      24624 non-null  object 
 6   G       24624 non-null  float64
 7   GS      18234 non-null  float64
 8   MP      24138 non-null  float64
 9   PER     24101 non-null  float64
 10  TS%     24538 non-null  float64
 11  3PAr    18840 non-null  float64
 12  FTr     24525 non-null  float64
 13  ORB%    20792 non-null  float64
 14  DRB%    20792 non-null  float64
 15  TRB%    21571 non-null  float64
 16  AST%    22555 non-null  float64
 17  STL%    20792 non-null  float64
 18  BLK%    20792 non-null  float64
 19  TOV%    19583 non-null  float64
 20  USG%    19641 non-null  float64
 21  OWS     24585 non-null  float64
 22

In [247]:
hof_df.isna().sum()

Year        67
HOF         12
Player      67
Pos         67
Age         75
Tm          67
G           67
GS        6457
MP         553
PER        590
TS%        153
3PAr      5851
FTr        166
ORB%      3899
DRB%      3899
TRB%      3120
AST%      2136
STL%      3899
BLK%      3899
TOV%      5108
USG%      5050
OWS        106
DWS        106
WS         106
WS/48      590
OBPM      3894
DBPM      3894
BPM       3894
VORP      3894
FG          67
FGA         67
FG%        166
3P        5763
3PA       5763
3P%       9274
2P          67
2PA         67
2P%        195
eFG%       166
FT          67
FTA         67
FT%        925
ORB       3894
DRB       3894
TRB        379
AST         67
STL       3894
BLK       3894
TOV       5045
PF          67
PTS         67
dtype: int64

# Data cleaning and EDA

In [248]:
hof_df = hof_df.dropna(subset=['HOF'])
hof_df.shape

(24679, 51)

In [249]:
class_column = 'HOF'
random_seed = 1000

X_train, X_test, y_train, y_test = train_test_split(hof_df.drop(columns=class_column), hof_df[class_column],
                                                   test_size=0.25, random_state=random_seed, stratify=hof_df[class_column])

Sanity Check

In [250]:
# X Train
print('On X train: ')
print('X train dimensions: ', X_train.shape)
display(X_train.head())

# X test
print('\nOn X test: ')
print('X test dimensions: ', X_test.shape)
display(X_test.head())

On X train: 
X train dimensions:  (18509, 50)


,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3634,1998.0,Cedric Ceballos,SF,28.0,TOT,47.0,25.0,990.0,19.3,0.560,...,0.738,75.0,146.0,221.0,60.0,33.0,16.0,72.0,88.0,536.0
23873,1969.0,Wali Jones,PG,26.0,PHI,81.0,NaN,2340.0,12.5,0.479,...,0.809,NaN,NaN,251.0,292.0,NaN,NaN,NaN,280.0,1071.0
20621,2008.0,Ryan Gomes,PF,25.0,MIN,82.0,74.0,2434.0,15.7,0.539,...,0.830,133.0,342.0,475.0,145.0,65.0,10.0,96.0,154.0,1033.0
8021,1977.0,Eric Money,PG,21.0,DET,73.0,NaN,1586.0,15.8,0.549,...,0.789,43.0,81.0,124.0,243.0,91.0,14.0,NaN,199.0,748.0
15361,1977.0,Louie Nelson,SG,25.0,SAS,4.0,NaN,57.0,8.4,0.527,...,0.571,2.0,5.0,7.0,3.0,2.0,0.0,NaN,9.0,18.0



On X test: 
X test dimensions:  (6170, 50)


,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
8064,2003.0,Eric Piatkowski,SG,32.0,LAC,62.0,26.0,1360.0,16.0,0.601,...,0.828,44.0,112.0,156.0,70.0,33.0,9.0,56.0,92.0,601.0
9307,1997.0,Greg Anthony,PG,29.0,VAN,65.0,44.0,1863.0,16.6,0.526,...,0.730,25.0,159.0,184.0,407.0,129.0,4.0,129.0,122.0,616.0
5065,1994.0,Dale Davis,PF,24.0,IND,66.0,64.0,2292.0,15.9,0.542,...,0.527,280.0,438.0,718.0,100.0,48.0,106.0,102.0,214.0,771.0
15148,2011.0,Linas Kleiza,SF,26.0,TOR,39.0,23.0,1032.0,10.1,0.502,...,0.631,37.0,140.0,177.0,40.0,19.0,7.0,65.0,99.0,436.0
5294,2011.0,Danilo Gallinari,SF,22.0,DEN,14.0,12.0,433.0,15.9,0.587,...,0.772,11.0,65.0,76.0,22.0,13.0,9.0,25.0,27.0,206.0


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [251]:
# X Train
print('On y train: ')
print('y train dimensions: ', y_train.shape)
display(y_train.head())

# X test
print('\nOn y test: ')
print('y test dimensions: ', y_test.shape)
display(y_test.head())

On y train: 
y train dimensions:  (18509,)


3634     No
23873    No
20621    No
8021     No
15361    No
Name: HOF, dtype: object


On y test: 
y test dimensions:  (6170,)


8064     No
9307     No
5065     No
15148    No
5294     No
Name: HOF, dtype: object

# Establish the training pipeline

In [257]:
# TODO: establish your own pipelines for the different data types
cat_pipeline = Pipeline(steps=[('cat_impute', SimpleImputer(strategy='most_frequent')),
                               ('onehot_cat', OneHotEncoder(handle_unknown='ignore'))])
num_pipeline = Pipeline(steps=[('impute_num', SimpleImputer(strategy='mean')),
                               ('scale_num', StandardScaler())])


In [258]:
# TODO: establish your preprocessing pipeline by your feature set
preprocess = ColumnTransformer([('cat_pipe', cat_pipeline, make_column_selector(dtype_include=object)),
                             ('num_pipe', num_pipeline, make_column_selector(dtype_include=np.number))],
                             remainder='passthrough')

print(preprocess)

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat_pipe',
                                 Pipeline(steps=[('cat_impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot_cat',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f95a538ef10>),
                                ('num_pipe',
                                 Pipeline(steps=[('impute_num',
                                                  SimpleImputer()),
                                                 ('scale_num',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f95a538ef90>)])


LogisticRegression

In [259]:
# TODO: set up your modeling pipeline
pipeLR = Pipeline(steps=[('preproc', preprocess),
                       ('mdl', LogisticRegression(penalty='elasticnet', solver='saga', tol=0.01))])


# Feel free to uncomment and edit the code below to visualize your overall pipeline
with config_context(display='diagram'):
    display(pipeLR)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_pipe',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot_cat',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f95a538ef10>),
                                                 ('num_pipe',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer()),
                                                                  ('scale_num',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f95a538ef90>)])),
                ('mdl',
                 LogisticRegression(penalty='elasticnet', solver='saga',
                                    tol=0.01))])

RandomForest

In [194]:
# TODO: set up your modeling pipeline RandomForest
pipeRandForest = Pipeline([('preproc', preprocess),
                        ('rf', RandomForestClassifier())])

# Feel free to uncomment and edit the code below to visualize your overall pipeline
with config_context(display='diagram'):
    display(pipeRandForest)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_pipe',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot_cat',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f95a761b990>),
                                                 ('num_pipe',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer()),
                                                                  ('scale_num',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f95a757d210>)])),
                ('rf', RandomForestClassifier())])

GradientBoost

In [195]:
# TODO: set up your modeling pipeline GradientBoosting
pipeGradBoost = Pipeline(steps=[('preproc', preprocess),
                       ('gbc', GradientBoostingClassifier())])


# Feel free to uncomment and edit the code below to visualize your overall pipeline
with config_context(display='diagram'):
    display(pipeGradBoost)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_pipe',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot_cat',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f95a761b990>),
                                                 ('num_pipe',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer()),
                                                                  ('scale_num',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f95a757d210>)])),
                ('gbc', GradientBoostingClassifier())])

## Cross-validation with hyperparameter tuning


LogisticalRegression

In [260]:
# TODO: set up your tuning grid 
# grid_search_lr.get_params().keys()
tuning_grid_lr = {'mdl__l1_ratio' : np.linspace(0,1,5),
                           'mdl__C': np.logspace(-1, 6, 3) }
grid_search_lr = GridSearchCV(pipeLR, param_grid = tuning_grid_lr, cv = 5, return_train_score=True)

print(grid_search_lr)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat_pipe',
                                                                         Pipeline(steps=[('cat_impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot_cat',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f95a538ef10>),
                                                                        ('num_pipe',
                                             

RandomForest

In [197]:
# TODO: set up your tuning grid
# grid_search_rf.get_params().keys()
tuning_grid_rf = param_grid = {'rf__n_estimators': [10, 50],
                                        'rf__max_features': ['auto', 'sqrt', 'log2'],
                                        'rf__max_depth' : [4,5,6],
                                        'rf__criterion' :['gini', 'entropy']
                                       }
grid_search_rf = GridSearchCV(pipeRandForest, param_grid = tuning_grid_rf, cv = 5, return_train_score=True)

GradientBoosting

In [198]:
# TODO: set up your tuning grid
# grid_search_gb.get_params().keys()
tuning_grid_gb = param_grid = {'gbc__learning_rate': [1, 0.5, 0.25],
                                       'gbc__n_estimators': [64, 128, 256],
                                       'gbc__max_depth': [2, 4],
                                       'gbc__min_samples_split': [2, 4]
                                      }
grid_search_gb = GridSearchCV(pipeGradBoost, param_grid = tuning_grid_gb, cv = 5, return_train_score=True)

# Fitting

In [ ]:
# Fit Logistic
grid_search_lr.fit(X_train, y_train)

In [ ]:
# Fit Gradient Boosting

#grid_search_gb.fit(X_train, y_train)

In [ ]:
# Fit RandomRainForest
grid_search_rf.fit(X_train, y_train)


# Best Performing Model Parameters

LogisticalRegression

In [ ]:
print("LogisticRegression best score:", grid_search_lr.best_score_)
grid_search_lr.best_params_

RandomForest

In [ ]:
print("RandomForest best score:", grid_search_rf.best_score_)
grid_search_rf.best_params_

# Final Fit

LogisticRegression

In [ ]:
grid_search_lr.best_estimator_

Random Forest

In [ ]:
grid_search_rf.best_estimator_

# Variable Importantance

LogisticRegression

In [ ]:
grid_search_lr.classes_

In [ ]:
vip_lr = grid_search_lr.best_estimator_['mdl'].coef_[0]
vip_lr

In [ ]:

#get names in correct preproc order
cat_names = grid_search_lr.best_estimator_.named_steps['preproc'].transformers_[0][1]
num_names = grid_search_lr.best_estimator_.named_steps['preproc'].transformers_[1][2]

#create df with vip info
print(cat_names)
print(num_names)
print(grid_search_lr.best_estimator_.named_steps['preproc'].transformers_)
coef_info = pd.DataFrame({'feat_names':np.hstack([cat_names, num_names]), 'vip': vip_lr})

#get sign and magnitude information
#coef_info = coef_info.assign(coef_mag = abs(coef_info['vip']),
 #                            coef_sign = np.sign(coef_info['vip']))

#sort and plot
#coef_info = coef_info.set_index('feat_names').sort_values(by='coef_mag', ascending=False)
#plt.figure(figsize = (17,15))
#sns.barplot(y=coef_info.index, x='coef_mag', hue='coef_sign', data=coef_info, orient='h', dodge=False)

Random Forest

In [ ]:
grid_search_rf.classes_

In [ ]:
vip_rf = grid_search_rf.best_estimator_['rf']
vip_rf

print(grid_search_rf)

In [ ]:
# TODO: feel free to uncomment and edit the code below to obtain your feature importance graph. 
# Did the graph surprise you? What features did you think were more important?


#get names in correct preproc order
cat_names = grid_search_rf.best_estimator_.named_steps['preproc'].transformers_[0][1]
num_names = grid_search_rf.best_estimator_.named_steps['preproc'].transformers_[1][2]

#create df with vip info
#print(cat_names)
#print(num_names)

coef_info = pd.DataFrame({'feat_names':np.hstack([cat_names, num_names]), 'vip': vip_rf})

#get sign and magnitude information
#coef_info = coef_info.assign(coef_mag = abs(coef_info['vip']),
                             #coef_sign = np.sign(coef_info['vip']))

#sort and plot
#coef_info = coef_info.set_index('feat_names').sort_values(by='coef_mag', ascending=False)
#plt.figure(figsize = (17,15))
#sns.barplot(y=coef_info.index, x='coef_mag', hue='coef_sign', data=coef_info, orient='h', dodge=False)